In [ ]:
import os
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.metrics import AUC, Precision, Recall
from sklearn.utils.class_weight import compute_class_weight

# 📁 Paths
BASE_DIR = "C:\\Users\\MANJU\\Desktop\\FYP_Moredata\\split_data"
TRAIN_DIR = os.path.join(BASE_DIR, "train")
VAL_DIR = os.path.join(BASE_DIR, "val")
TEST_DIR = os.path.join(BASE_DIR, "test")

# 🔢 Parameters
IMG_HEIGHT, IMG_WIDTH = 224, 224
BATCH_SIZE = 64
EPOCHS = 30

# 🧪 Data Augmentation
train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rotation_range=45,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.3,
    shear_range=0.3,
    brightness_range=[0.6, 1.4],
    horizontal_flip=True,
    vertical_flip=True,
    fill_mode='nearest'
)

val_test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

# 📦 Data Loaders
train_generator = train_datagen.flow_from_directory(
    TRAIN_DIR,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='binary'
)

val_generator = val_test_datagen.flow_from_directory(
    VAL_DIR,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='binary'
)

test_generator = val_test_datagen.flow_from_directory(
    TEST_DIR,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='binary',
    shuffle=False
)

# 📏 Class Weight Calculation
class_weights = compute_class_weight(
    class_weight='balanced',
    classes=np.unique(train_generator.classes),
    y=train_generator.classes
)
class_weights = dict(enumerate(class_weights))

# 🧠 Load MobileNetV2 Base
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(IMG_HEIGHT, IMG_WIDTH, 3))
base_model.trainable = False  # Initially freeze base model

# 🏗️ Custom Classifier Head
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(64, activation='relu')(x)
x = Dropout(0.3)(x)
output = Dense(1, activation='sigmoid')(x)

model = Model(inputs=base_model.input, outputs=output)

# 🧪 Compile Model
model.compile(
    optimizer=Adam(learning_rate=0.0003),
    loss='binary_crossentropy',
    metrics=['accuracy', AUC(name='auc'), Precision(name='precision'), Recall(name='recall')]
)

# 📌 Callbacks
checkpoint = ModelCheckpoint("best_mobilenetv2_model.h5", monitor='val_accuracy', save_best_only=True, verbose=1)
early_stop = EarlyStopping(monitor='val_loss', patience=7, restore_best_weights=True)
lr_scheduler = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, verbose=1)

# 🚀 Phase 1: Train Custom Head
history = model.fit(
    train_generator,
    steps_per_epoch=len(train_generator),
    epochs=EPOCHS,
    validation_data=val_generator,
    validation_steps=len(val_generator),
    class_weight=class_weights,
    callbacks=[checkpoint, early_stop, lr_scheduler]
)

# 🔓 Phase 2: Unfreeze and Fine-tune MobileNetV2
base_model.trainable = True

# Re-compile with lower LR
model.compile(
    optimizer=Adam(learning_rate=1e-5),
    loss='binary_crossentropy',
    metrics=['accuracy', AUC(name='auc'), Precision(name='precision'), Recall(name='recall')]
)

# Fine-tuning Training
history_finetune = model.fit(
    train_generator,
    steps_per_epoch=len(train_generator),
    epochs=10,
    validation_data=val_generator,
    validation_steps=len(val_generator),
    class_weight=class_weights,
    callbacks=[checkpoint, early_stop, lr_scheduler]
)

# 🧾 Evaluate Model
loss, accuracy, auc, precision, recall = model.evaluate(test_generator, steps=len(test_generator))
print(f"\n✅ Final Test Accuracy: {accuracy * 100:.2f}%")
print(f"📈 AUC: {auc:.4f} | 🎯 Precision: {precision:.4f} | 🔍 Recall: {recall:.4f}")

# 💾 Save Final Model
model.save("final_mobilenetv2_model.h5")
print("✅ Optimized MobileNetV2 training complete!")


Found 226 images belonging to 2 classes.
Found 48 images belonging to 2 classes.
Found 50 images belonging to 2 classes.


C:\Users\MANJU\AppData\Roaming\Python\Python311\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.6276 - auc: 0.5994 - loss: 0.7237 - precision: 0.3122 - recall: 0.4818
Epoch 1: val_accuracy improved from -inf to 0.56250, saving model to best_mobilenetv2_model.h5


4/4 ━━━━━━━━━━━━━━━━━━━━ 22s 4s/step - accuracy: 0.6242 - auc: 0.5962 - loss: 0.7296 - precision: 0.3077 - recall: 0.4640 - val_accuracy: 0.5625 - val_auc: 0.6539 - val_loss: 0.6641 - val_precision: 0.3043 - val_recall: 0.5833 - learning_rate: 3.0000e-04
Epoch 2/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.6075 - auc: 0.6028 - loss: 0.7099 - precision: 0.3028 - recall: 0.5323
Epoch 2: val_accuracy did not improve from 0.56250
4/4 ━━━━━━━━━━━━━━━━━━━━ 8s 2s/step - accuracy: 0.6063 - auc: 0.6034 - loss: 0.7122 - precision: 0.3068 - recall: 0.5366 - val_accuracy: 0.5000 - val_auc: 0.7465 - val_loss: 0.7003 - val_precision: 0.3000 - val_recall: 0.7500 - learning_rate: 3.0000e-04
Epoch 3/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.5924 - auc: 0.6592 - loss: 0.6911 - precision: 0.3484 - recall: 0.6680
Epoch 3: val_accuracy did not improve from 0.56250
4/4 ━━━━━━━━━━━━━━━━━━━━ 8s 2s/step - accuracy: 0.5943 - auc: 0.6678 - loss: 0.6816 - precision: 0.3489 - recall: 0.6772 - va

4/4 ━━━━━━━━━━━━━━━━━━━━ 9s 3s/step - accuracy: 0.7224 - auc: 0.8236 - loss: 0.5294 - precision: 0.4841 - recall: 0.6839 - val_accuracy: 0.6250 - val_auc: 0.8137 - val_loss: 0.5800 - val_precision: 0.3750 - val_recall: 0.7500 - learning_rate: 3.0000e-04
Epoch 5/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.7725 - auc: 0.8395 - loss: 0.4975 - precision: 0.5324 - recall: 0.7894
Epoch 5: val_accuracy improved from 0.62500 to 0.68750, saving model to best_mobilenetv2_model.h5


4/4 ━━━━━━━━━━━━━━━━━━━━ 9s 3s/step - accuracy: 0.7693 - auc: 0.8386 - loss: 0.4982 - precision: 0.5272 - recall: 0.7708 - val_accuracy: 0.6875 - val_auc: 0.8356 - val_loss: 0.5488 - val_precision: 0.4286 - val_recall: 0.7500 - learning_rate: 3.0000e-04
Epoch 6/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.7761 - auc: 0.8342 - loss: 0.5095 - precision: 0.5467 - recall: 0.7696
Epoch 6: val_accuracy did not improve from 0.68750
4/4 ━━━━━━━━━━━━━━━━━━━━ 8s 2s/step - accuracy: 0.7766 - auc: 0.8341 - loss: 0.5092 - precision: 0.5449 - recall: 0.7693 - val_accuracy: 0.6458 - val_auc: 0.8484 - val_loss: 0.5697 - val_precision: 0.3913 - val_recall: 0.7500 - learning_rate: 3.0000e-04
Epoch 7/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.7903 - auc: 0.8646 - loss: 0.4557 - precision: 0.5450 - recall: 0.7824
Epoch 7: val_accuracy did not improve from 0.68750
4/4 ━━━━━━━━━━━━━━━━━━━━ 10s 2s/step - accuracy: 0.7854 - auc: 0.8615 - loss: 0.4597 - precision: 0.5398 - recall: 0.7723 - va

4/4 ━━━━━━━━━━━━━━━━━━━━ 9s 3s/step - accuracy: 0.8124 - auc: 0.8848 - loss: 0.4511 - precision: 0.6303 - recall: 0.7977 - val_accuracy: 0.7083 - val_auc: 0.8762 - val_loss: 0.4645 - val_precision: 0.4500 - val_recall: 0.7500 - learning_rate: 1.5000e-04
Epoch 17/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.8077 - auc: 0.8934 - loss: 0.4052 - precision: 0.5536 - recall: 0.9240
Epoch 17: val_accuracy did not improve from 0.70833
4/4 ━━━━━━━━━━━━━━━━━━━━ 8s 2s/step - accuracy: 0.8063 - auc: 0.8920 - loss: 0.4073 - precision: 0.5561 - recall: 0.9070 - val_accuracy: 0.7083 - val_auc: 0.8750 - val_loss: 0.4667 - val_precision: 0.4500 - val_recall: 0.7500 - learning_rate: 1.5000e-04
Epoch 18/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.8367 - auc: 0.9004 - loss: 0.3865 - precision: 0.6291 - recall: 0.8592
Epoch 18: val_accuracy improved from 0.70833 to 0.72917, saving model to best_mobilenetv2_model.h5



Epoch 18: ReduceLROnPlateau reducing learning rate to 7.500000356230885e-05.
4/4 ━━━━━━━━━━━━━━━━━━━━ 8s 2s/step - accuracy: 0.8304 - auc: 0.8971 - loss: 0.3933 - precision: 0.6186 - recall: 0.8481 - val_accuracy: 0.7292 - val_auc: 0.8773 - val_loss: 0.4794 - val_precision: 0.4800 - val_recall: 1.0000 - learning_rate: 1.5000e-04
Epoch 19/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.8425 - auc: 0.9224 - loss: 0.3431 - precision: 0.6062 - recall: 0.8405 
Epoch 19: val_accuracy did not improve from 0.72917
4/4 ━━━━━━━━━━━━━━━━━━━━ 8s 2s/step - accuracy: 0.8377 - auc: 0.9169 - loss: 0.3559 - precision: 0.6061 - recall: 0.8260 - val_accuracy: 0.7292 - val_auc: 0.8854 - val_loss: 0.4836 - val_precision: 0.4800 - val_recall: 1.0000 - learning_rate: 7.5000e-05
Epoch 20/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.7996 - auc: 0.9186 - loss: 0.3704 - precision: 0.5638 - recall: 0.8901 
Epoch 20: val_accuracy did not improve from 0.72917
4/4 ━━━━━━━━━━━━━━━━━━━━ 7s 2s/step - accu


✅ Final Test Accuracy: 74.00%
📈 AUC: 0.9035 | 🎯 Precision: 0.4800 | 🔍 Recall: 1.0000
✅ Optimized MobileNetV2 training complete!
